In [1]:
pip install selenium

In [2]:
pip install chromedriver-autoinstaller

Note: you may need to restart the kernel to use updated packages.


In [4]:
#일단 맨 아래까지 긁어옴
import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

from selenium.webdriver.chrome.options import Options
import chromedriver_autoinstaller
import subprocess
import shutil
from time import sleep 

try:
    shutil.rmtree(r"c:\chrometemp")  #쿠키 / 캐쉬파일 삭제
except FileNotFoundError:
    pass

subprocess.Popen(r'C:\Program Files\Google\Chrome\Application\chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\chrometemp"') # 디버거 크롬 구동
url = 'https://www.fragrantica.com/perfume/Maison-Francis-Kurkdjian/Baccarat-Rouge-540-33519.html'

option = Options()
option.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]
try:
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)
except:
    chromedriver_autoinstaller.install(True)
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)
#---------------------------------------------------------------------------디버깅 크롤링
#디버거 크롤링(일반 크롬과 똑같은 설정으로 셀레니움을 접속시키는것)을 하기 위한세팅 - 이걸 하는 이유는 사이트가 봇탐지 해놔서 우회해서 크롤링 하기 위해
#봇이 접속시키는게 아니라 일반 사용자처럼 해서 우회해서 접속 시키겠다는 의미

#사이트 접속(female)
driver.get(url) #사이트 접속
driver.implicitly_wait(5) # 위 구문 실행뒤에 다음 구문 바로 실행하면 오류 나기 때문에 기다려 주는것
# #spring 선택 
clickBtn = driver.find_element_by_css_selector('#offCanvasLeftOverlap1 > div > div > div:nth-child(2) > div.ais-Panel-body > p > div > select > option:nth-child(3)')
clickBtn.click() #버튼 클릭
sleep(3) #3초 기다려줌
#첫번째 show more result 버튼 클릭
clickRS = driver.find_element_by_css_selector('#main-content > div.grid-x.grid-margin-x > div.small-12.medium-8.large-9.cell > div > div > div > div.off-canvas-content.content1 > div.grid-x.grid-padding-x.grid-padding-y > div > div:nth-child(3) > div > div > div > div > div > button')
clickRS.send_keys(Keys.ENTER) 
sleep(3)
#show more result 버튼 끝까지 클릭  *위에 한번 따로 클릭한 이유는 처음 누를때 선택자랑 두번째부터 마지막까지 선택자가 다름!
while True:
    try:
        clickRS = driver.find_element_by_css_selector('#main-content > div.grid-x.grid-margin-x > div.small-12.medium-8.large-9.cell > div > div > div > div.off-canvas-content.content1.has-reveal-left > div.grid-x.grid-padding-x.grid-padding-y > div > div:nth-child(3) > div > div > div > div > div > button')
        clickRS.send_keys(Keys.ENTER)
        sleep(3)
    except:
        print('끝')
        break


C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_17732/2163175586.py:32: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)


In [ ]:
#DB저장하는것까지 포함 
#최종완성본
# href를 긁어올 선택자 만들기 

a = []
for ggg in range(1, 1500): 
    a.append('//*[@id="main-content"]/div[1]/div[1]/div/div/div/div[2]/div[1]/div/div[3]/div/div/div/span/div['+ str(ggg)+']/div[2]/p[1]/a')

# href 긁어오기
hrefList = []
for gg in range(len(a)):
    hrefTemp = driver.find_elements_by_xpath(a[gg])
    try:
        hrefList.append(hrefTemp[0].get_attribute("href"))
    except:
        print('끝')
        break

p_info=[]
for g in range(len(hrefList)):
    driver.get(hrefList[g])
    sleep(42)
    try:         
        topNote = driver.find_element_by_xpath('//*[@id="pyramid"]/div[1]/div/div[2]/div[3]/div').text # 선택자 칸에 있는 text다 가져오기
        toplist = []
        topNoteNum =[]
        toplist.append(topNote) #text를 list에 담기
        toplist = toplist[0].split("\n") # ['Saffron\nJasmine'] list에 담으면 공백이 '\n'로 표기 때문에 자름
        for i in range(len(toplist)) :
            if(len(toplist) == 1) : # list길이가 하나면 노트가 하나 있는것이기 때문에 하나 있을때의 노트 xpath를 가져와 노트 번호 구함
                topNoteN = driver.find_element_by_xpath('//*[@id="pyramid"]/div[1]/div/div[2]/div[3]/div/div/div[2]/a')
                topNum = topNoteN.get_attribute("href").split('/')[4].split('-')[-1].split('.')[0]
            else :
                r = i+1  # list길이가 하나이상일 경우  xpath가 1씩 증가함 
                topNoteN = driver.find_element_by_xpath('//*[@id="pyramid"]/div[1]/div/div[2]/div[3]/div/div['+ str(r)+']/div[2]/a')
                topNoteNu = topNoteN.get_attribute("href").split('/')[4].split('-')[-1].split('.')[0]
                topNoteNum.append(topNoteNu)
                topNum = "-".join(topNoteNum) # 번호는 '-'로 붙여서 topnum에 저장 
        #노트 이름 저장
        #아래 노트들에는 이미 -가 이름이들어가 있기때문에 '-' 공백(\n)으로 변경하고 저장함 나중에 '-'기준으로 잘라 써야 되기 때문에
        if "Lily-of-the-Valley" or "Umbu-Cajá | Taperebá" or "Pua keni keni (Pua-lulu)" or "Karo-Karounde" or "Ylang-Ylang" or "Para-Cresyl Phenyl Acetate" in topNote :
            topNote = topNote.replace('\n','-')
            topNote = topNote.replace('Lily-of-the-Valley' , 'Lily of the Valley')
            topNote = topNote.replace('Umbu-Cajá | Taperebá' , 'Umbu Cajá | Taperebá')
            topNote = topNote.replace('Pua keni keni (Pua-lulu)' , 'Pua keni keni (Pua lulu)')
            topNote = topNote.replace('Karo-Karounde' , 'Karo Karounde')
            topNote = topNote.replace('Ylang-Ylang' , 'Ylang Ylang')
            topNote = topNote.replace('Para-Cresyl Phenyl Acetate' , 'Para Cresyl Phenyl Acetate')
        else : 
            topNote = topNote.replace('\n','-') # 위의 노트 이름이 없을 경우 빈공간을 '-'로 변경하여 topNote(노트이름)dmfh wjwkd
    except:#노트가 없을경우 예외 처리
        topNote = ''
        topNum = ''
        print('topNote / Num 없음')
        

    try:
        middleNote = driver.find_element_by_xpath('//*[@id="pyramid"]/div[1]/div/div[2]/div[4]/div').text
        middlelist = []
        middleNoteNum =[]
        middlelist.append(middleNote)
        middlelist = middlelist[0].split("\n")
        for ii in range(len(middlelist)) :
            if(len(middlelist) == 1) : 
                middleNoteN = driver.find_element_by_xpath('//*[@id="pyramid"]/div[1]/div/div[2]/div[4]/div/div/div[2]/a')
                middleNoteNu = middleNoteN.get_attribute("href").split('/')[4].split('-')[-1].split('.')[0]
                middleNoteNum.append(middleNoteNu)
                middleNum = "-".join(middleNoteNum)
            else:
                rr = ii+1
                middleNoteN = driver.find_element_by_xpath('//*[@id="pyramid"]/div[1]/div/div[2]/div[4]/div/div['+ str(rr)+']/div[2]/a')
                middleNoteNu = middleNoteN.get_attribute("href").split('/')[4].split('-')[-1].split('.')[0]
                middleNoteNum.append(middleNoteNu)
                middleNum = "-".join(middleNoteNum)
        if "Lily-of-the-Valley" or "Umbu-Cajá | Taperebá" or "Pua keni keni (Pua-lulu)" or "Karo-Karounde" or "Ylang-Ylang" or "Para-Cresyl Phenyl Acetate" in middleNote :
            middleNote = middleNote.replace('\n','-')
            middleNote = middleNote.replace('Lily-of-the-Valley' , 'Lily of the Valley')
            middleNote = middleNote.replace('Umbu-Cajá | Taperebá' , 'Umbu Cajá | Taperebá')
            middleNote = middleNote.replace('Pua keni keni (Pua-lulu)' , 'Pua keni keni (Pua lulu)')
            middleNote = middleNote.replace('Karo-Karounde' , 'Karo Karounde')
            middleNote = middleNote.replace('Ylang-Ylang' , 'Ylang Ylang')
            middleNote = middleNote.replace('Para-Cresyl Phenyl Acetate' , 'Para Cresyl Phenyl Acetate')
        else : 
            middleNote = middleNote.replace('\n','-')
    except:
        middleNote = ''
        middleNum = ''
        print('middleNote / Num 없음')
                
    try:
        baseNote = driver.find_element_by_xpath('//*[@id="pyramid"]/div[1]/div/div[2]/div[5]/div').text
        baselist = []
        baseNoteNum =[]
        baselist.append(baseNote)
        baselist = baselist[0].split("\n")
        for iii in range(len(baselist)) :
            if(len(baselist) == 1) : 
                baseNoteN = driver.find_element_by_xpath('//*[@id="pyramid"]/div[1]/div/div[2]/div[5]/div/div/div[2]/a')
                baseNoteNu = baseNoteN.get_attribute("href").split('/')[4].split('-')[-1].split('.')[0]
                baseNoteNum.append(baseNoteNu)
                baseNum = "-".join(baseNoteNum)
            else :
                rrr = iii+1
                baseNoteN = driver.find_element_by_xpath('//*[@id="pyramid"]/div[1]/div/div[2]/div[5]/div/div['+ str(rrr)+']/div[2]/a')
                baseNoteNu = baseNoteN.get_attribute("href").split('/')[4].split('-')[-1].split('.')[0]
                baseNoteNum.append(baseNoteNu)
                baseNum = "-".join(baseNoteNum)
        if "Lily-of-the-Valley" or "Umbu-Cajá | Taperebá" or "Pua keni keni (Pua-lulu)" or "Karo-Karounde" or "Ylang-Ylang" or "Para-Cresyl Phenyl Acetate" in baseNote :
            baseNote = baseNote.replace('\n','-')
            baseNote = baseNote.replace('Lily-of-the-Valley' , 'Lily of the Valley')
            baseNote = baseNote.replace('Umbu-Cajá | Taperebá' , 'Umbu Cajá | Taperebá')
            baseNote = baseNote.replace('Pua keni keni (Pua-lulu)' , 'Pua keni keni (Pua lulu)')
            baseNote = baseNote.replace('Karo-Karounde' , 'Karo Karounde')
            baseNote = baseNote.replace('Ylang-Ylang' , 'Ylang Ylang')
            baseNote = baseNote.replace('Para-Cresyl Phenyl Acetate' , 'Para Cresyl Phenyl Acetate')
        else : 
            baseNote = baseNote.replace('\n','-')
    except:
        baseNote = ''
        baseNum = ''
        print('baseNote / Num 없음')
    
  
    
    
    #href예시) - https://www.fragrantica.com/perfume/Maison-Francis-Kurkdjian/Baccarat-Rouge-540-33519.html를 '/'이거 기준으로 자를거임
    href = hrefList[g].split('/')
    brandNM = href[4]# 4번째 인덱스가 브랜드이름
    pTemp = str(href[5]).split('.')# 5번째 인덱스에서 .HTML지우기 위해서
    pTemp = pTemp[0].split('-') # 뒤에서 첫번째까 향수 번호
    fNo = pTemp[-1] # 뒤에서 첫번째까 향수 번호
    fNM = pTemp[:-1]# 맨뒤에거 뺀 전부가 향수 이름 
    fNM =  " ".join(fNM)# 잘려져 있는 향수 이름을 띄어쓰기로 붙여줌
    f_number = int(fNo)#향수 번호 int로 변경 db에 number로 설정함
    try :
        brand = driver.find_element_by_xpath('//*[@id="main-content"]/div[1]/div[1]/div/div[2]/div[1]/div[2]/p/a/img')
        brandNo = brand.get_attribute("src").split('/')[5].split('.')[-2] #브랜드 로고 IMG Xpath 에서 src부분에 있는 브랜드 번호 가져옴
        d_number = int(brandNo) #향수 번호 int로 변경 db에 number로 설정함
    except :#브랜드 없을 경우 예외 처리
        brandNo = ''
        print('brandNo 없음')
        

    Pic = driver.find_element_by_xpath('//*[@id="main-content"]/div[1]/div[1]/div/div[2]/div[1]/div[1]/div/div/img')
    fPic = Pic.get_attribute("src") # 사진 주소 긁어오기 
    #INSERT할 정보 튜플에 담고 LIST에 담기
    lists = [(f_number, fNM, topNote , middleNote , baseNote  , d_number , fPic , topNum , middleNum , baseNum)]
    
    #향수가 여러 계절에 겹쳐서 들어간 경우가 있어서 select로 검사하고 있을시 계절을 업데이트 해주고 아닐경우 인서트 하는작업
    sql_a = "select count(*) from tbl_perfume where f_number ="+str(f_number)
    cursor.execute(sql_a)

    for y in cursor :
        n = list(y)
        nn = n[0]
        print(nn)
        if ( nn == 0 ) :
            import cx_Oracle
            import os
            LOCATION = r"C:\instantclient_21_3"
            os.environ["PATH"] = LOCATION + ";" + os.environ["PATH"] #환경변수 등록
            connect_b = cx_Oracle.connect("final05", "final", "masternull.iptime.org:1521/orcl")
            cursor_b = connect_b.cursor()
            sql_b = "insert into tbl_perfume values( :1 , :2 , :3 , :4 , :5 ,  2 , perfume_seq.nextval ,:6 , :7 , :8 , :9 , :10 ,'4')"
            for u in lists : 
                cursor_b.execute(sql_b , u)
                cursor_b.close()
                connect_b.commit()
                
                print(u)
        if ( nn == 1 ) :
            import cx_Oracle
            import os
            LOCATION = r"C:\instantclient_21_3"
            os.environ["PATH"] = LOCATION + ";" + os.environ["PATH"] #환경변수 등록
            connect_c = cx_Oracle.connect("final05", "final", "masternull.iptime.org:1521/orcl")
            cursor_c = connect_c.cursor()
            sql_c = "select season from tbl_perfume where f_number ="+str(f_number)
            cursor_c.execute(sql_c)
            print("except" , f_number)
            for t in cursor_c :
                if( t != "") :
                    s = list(t)
                    ss  = s[0]
                    print(ss)
                    if( ss == '1' ) :
                        import cx_Oracle
                        import os
                        LOCATION = r"C:\instantclient_21_3"
                        os.environ["PATH"] = LOCATION + ";" + os.environ["PATH"] #환경변수 등록
                        connect_d = cx_Oracle.connect("final05", "final", "masternull.iptime.org:1521/orcl")
                        cursor_d = connect_d.cursor()
                        sql_d = "update tbl_perfume set season = '1-4' where f_number = " +str(f_number)
                        cursor_d.execute(sql_d)
                        cursor_d.close()
                        connect_d.commit()
                        print('1-4')
                    elif( ss == '2' ) :
                        import cx_Oracle
                        import os
                        LOCATION = r"C:\instantclient_21_3"
                        os.environ["PATH"] = LOCATION + ";" + os.environ["PATH"] #환경변수 등록
                        connect_e = cx_Oracle.connect("final05", "final", "masternull.iptime.org:1521/orcl")
                        cursor_e = connect_e.cursor()
                        sql_e = "update tbl_perfume set season = '2-4' where f_number = " +str(f_number)
                        cursor_e.execute(sql_e)
                        cursor_e.close()
                        connect_e.commit()
                        print('2-4')
                    elif( ss == '3' ) :
                        import cx_Oracle
                        import os
                        LOCATION = r"C:\instantclient_21_3"
                        os.environ["PATH"] = LOCATION + ";" + os.environ["PATH"] #환경변수 등록
                        connect_f = cx_Oracle.connect("final05", "final", "masternull.iptime.org:1521/orcl")
                        cursor_f = connect_f.cursor()
                        sql_f = "update tbl_perfume set season = '3-4' where f_number = " +str(f_number)
                        cursor_f.execute(sql_f)
                        cursor_f.close()
                        connect_f.commit()
                        print('3-4')
                    elif( ss == '1-2') :
                        import cx_Oracle
                        import os
                        LOCATION = r"C:\instantclient_21_3"
                        os.environ["PATH"] = LOCATION + ";" + os.environ["PATH"] #환경변수 등록
                        connect_g = cx_Oracle.connect("final05", "final", "masternull.iptime.org:1521/orcl")
                        cursor_g = connect_g.cursor()
                        sql_g = "update tbl_perfume set season = '1-2-4' where f_number = " +str(f_number)
                        cursor_g.execute(sql_g)
                        cursor_g.close()
                        connect_g.commit()
                        print('1-2-4')
                    elif( ss == '1-3') :
                        import cx_Oracle
                        import os
                        LOCATION = r"C:\instantclient_21_3"
                        os.environ["PATH"] = LOCATION + ";" + os.environ["PATH"] #환경변수 등록
                        connect_h = cx_Oracle.connect("final05", "final", "masternull.iptime.org:1521/orcl")
                        cursor_h = connect_h.cursor()
                        sql_h = "update tbl_perfume set season = '1-3-4' where f_number = " +str(f_number)
                        cursor_h.execute(sql_h)
                        cursor_h.close()
                        connect_g.commit()
                        print('1-3-4')
                    elif( ss == '2-3') :
                        import cx_Oracle
                        import os
                        LOCATION = r"C:\instantclient_21_3"
                        os.environ["PATH"] = LOCATION + ";" + os.environ["PATH"] #환경변수 등록
                        connect_i = cx_Oracle.connect("final05", "final", "masternull.iptime.org:1521/orcl")
                        cursor_i = connect_i.cursor()
                        sql_i = "update tbl_perfume set season = '2-3-4' where f_number = " +str(f_number)
                        cursor_i.execute(sql_i)
                        cursor_i.close()
                        connect_i.commit()
                        print('2-3-4')
                    elif( ss == '1-2-3') :
                        import cx_Oracle
                        import os
                        LOCATION = r"C:\instantclient_21_3"
                        os.environ["PATH"] = LOCATION + ";" + os.environ["PATH"] #환경변수 등록
                        connect_j = cx_Oracle.connect("final05", "final", "masternull.iptime.org:1521/orcl")
                        cursor_j = connect_j.cursor()
                        sql_g = "update tbl_perfume set season = '1-2-3-4' where f_number = " +str(f_number)
                        cursor_j.execute(sql_j)
                        cursor_j.close()
                        connect_j.commit()
                        print('1-2-3-4')
                    else :
                        print("gender = 2 season = 4  이미 있는 향수")
    
connect.commit()    

   
    
   